# Research for Manual:
Last year, here were the different assets and their associated profits: \
{'Refrigerators': 0.020713448079427082,\
 'Earrings': 0.12367402886284722,\
 'Blankets': -0.32888405330882353,\
 'Sleds': -0.2829537109375,\
 'Sculptures': 0.19637428385416666,\
 'PS6': 0.3095545703125,\
 'Serum': -0.8157531666666666,\
 'Lamps': 6.103515625e-05,\
 'Chocolate': -0.000404595947265625} \ \ \ 
 I found decent mappings between a few of the assets, this year in terms of the news stories. \
* Statues - Red flag\
* E blankets - Needles\
* PS6 - VR\
* Ranch - Earrings\
* Fridges - Haystacks\
* Dog Sled - Solar Panels 
I categorised based on the last year returns on the mappings, and the rest purely on vibe, using a system of + and - just to reduce overfitting in the data.
 

In [ ]:
import numpy as np
import cvxpy as cp
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
print(cp.installed_solvers())

In [ ]:
sentiments = {
    'Haystacks': '+',
    'Ranch': '++',
    'Needles': '----',
    'SolarPanels': '---',
    'RedFlags': '++',
    'VR': '+++',
    'CosmicClean': '-',
    'Shirts': '+',
    'Coffee': '--'
}
returns = {
    '+': 0.02,
    '++': 0.15,
    '+++': 0.30,
    '-': -0.02,
    '--': -0.15,
    '---': -0.3,
    '----': -0.7
}

products = list(sentiments.keys())

In [ ]:

rets = np.array([returns[sentiments[products[i-1]]] for i in range(1,10)])
pi = cp.Variable(9)
objective = cp.Minimize(120 * cp.sum_squares(pi) - 10000 * rets.T @ pi)
constraints = [cp.norm(pi, 1) <= 100]
prob = cp.Problem(objective, constraints)

prob.solve()
print('Optimal allocation without integer constraints:')
for i in range(9):
    print("Position in ", products[i], ': ', f"{pi.value[i]:,.2f}", '%', sep='')

In [ ]:
s1 = ' + '.join(['('+str(returns[sentiments[products[i-1]]])+')*p'+str(i)+'*10000-120*(p'+str(i)+')^2' for i in range(1,10)])
s2 = ' + '.join(['Abs[p'+str(i)+']' for i in range(1,10)]) + '<=100,'
s3 = ', '.join(['Element[p'+str(i)+', Integers]' for i in range(1,10)])
s4 = ', '.join(['p'+str(i) for i in range(1,10)])

In [ ]:
with  WolframLanguageSession() as session:
    session.start()
    val_max, sol = session.evaluate(wlexpr('NMaximize[{'+s1+','+s2+s3+'}, {'+s4+'}]'))

In [ ]:
print("Maximum profit achievable:", val_max)
print("Percentage of capital used: ", sum([abs(el[1]) for el in sol]), '%', sep='')

In [ ]:
for i in range(9):
    print("Position in ", products[i], ': ', sol[i][1], '%', sep='')